In [1]:
import gurobipy as gp
from gurobipy import GRB
import random

# Scale up the problem size
num_suppliers = 5
num_factories = 5
num_warehouses = 5
num_customers = 5
num_time_periods = 5

suppliers = range(1, num_suppliers + 1)
factories = range(1, num_factories + 1)
warehouses = range(1, num_warehouses + 1)
customers = range(1, num_customers + 1)
time_periods = range(1, num_time_periods + 1)

# Generate random data
random.seed(42)

c_hi = {(h, i): random.randint(5, 15) for h in suppliers for i in factories}
c_ie = {(i, e): random.randint(3, 10) for i in factories for e in warehouses}
c_ej = {(e, j): random.randint(2, 8) for e in warehouses for j in customers}
fixed_cost_factory = {i: random.randint(300, 600) for i in factories}
fixed_cost_warehouse = {e: random.randint(200, 400) for e in warehouses}
supply_capacity = {h: random.randint(100, 200) for h in suppliers}
factory_capacity = {i: random.randint(100, 200) for i in factories}
warehouse_capacity = {e: random.randint(100, 200) for e in warehouses}
demand = {(j, t): random.randint(20, 50) for j in customers for t in time_periods}
initial_inventory = {e: random.randint(0, 50) for e in warehouses}
holding_cost = {e: random.uniform(1, 2) for e in warehouses}
budget = 5000

# Initialize model
model = gp.Model("Scaled_Facility_Location")

# Decision variables
y_i = model.addVars(factories, vtype=GRB.BINARY, name="open_factory")
y_e = model.addVars(warehouses, vtype=GRB.BINARY, name="open_warehouse")
x_hi = model.addVars(c_hi.keys(), vtype=GRB.CONTINUOUS, name="flow_supplier_factory")
x_ie = model.addVars(c_ie.keys(), vtype=GRB.CONTINUOUS, name="flow_factory_warehouse")
x_ejt = model.addVars([(e, j, t) for e in warehouses for j in customers for t in time_periods],
                      vtype=GRB.CONTINUOUS, name="flow_warehouse_customer")
inventory_et = model.addVars([(e, t) for e in warehouses for t in time_periods],
                             vtype=GRB.CONTINUOUS, name="inventory")

# Objective function
transport_cost = gp.quicksum(c_hi[h, i] * x_hi[h, i] for h, i in c_hi) + \
                 gp.quicksum(c_ie[i, e] * x_ie[i, e] for i, e in c_ie) + \
                 gp.quicksum(c_ej[e, j] * x_ejt[e, j, t] for e, j, t in x_ejt.keys())
fixed_costs = gp.quicksum(fixed_cost_factory[i] * y_i[i] for i in factories) + \
              gp.quicksum(fixed_cost_warehouse[e] * y_e[e] for e in warehouses)
holding_costs = gp.quicksum(holding_cost[e] * inventory_et[e, t] for e, t in inventory_et.keys())

model.setObjective(transport_cost + fixed_costs + holding_costs, GRB.MINIMIZE)

# Constraints
# Budget constraint
model.addConstr(fixed_costs + transport_cost <= budget, name="Budget")

# Supply constraints
for h in suppliers:
    model.addConstr(gp.quicksum(x_hi[h, i] for i in factories) <= supply_capacity[h], name=f"Supply_{h}")

# Factory capacity constraints
for i in factories:
    model.addConstr(gp.quicksum(x_ie[i, e] for e in warehouses) <= factory_capacity[i] * y_i[i], name=f"Factory_{i}")

# Warehouse capacity constraints and inventory balance
for e in warehouses:
    for t in time_periods:
        inflow = gp.quicksum(x_ie[i, e] for i in factories)
        outflow = gp.quicksum(x_ejt[e, j, t] for j in customers)
        prev_inventory = inventory_et[e, t - 1] if t > 1 else initial_inventory[e]
        model.addConstr(prev_inventory + inflow - outflow == inventory_et[e, t], name=f"Inventory_Balance_{e}_{t}")
        model.addConstr(inventory_et[e, t] <= warehouse_capacity[e] * y_e[e], name=f"Warehouse_Capacity_{e}_{t}")

# Demand satisfaction
for j in customers:
    for t in time_periods:
        model.addConstr(gp.quicksum(x_ejt[e, j, t] for e in warehouses) == demand[j, t], name=f"Demand_{j}_{t}")

# Solve the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal Cost:", model.ObjVal)
    print("Open Factories:", {i: y_i[i].X for i in factories})
    print("Open Warehouses:", {e: y_e[e].X for e in warehouses})
    print("Flows (Supplier-Factory):", {key: x_hi[key].X for key in c_hi.keys()})
    print("Flows (Factory-Warehouse):", {key: x_ie[key].X for key in c_ie.keys()})
    print("Flows (Warehouse-Customer):", {key: x_ejt[key].X for key in x_ejt.keys()})
    print("Inventory Levels:", {key: inventory_et[key].X for key in inventory_et.keys()})
else:
    print("No optimal solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-26
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 86 rows, 210 columns and 710 nonzeros
Model fingerprint: 0x1a9bb57f
Variable types: 200 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 5e+03]
Presolve removed 5 rows and 25 columns
Presolve time: 0.02s
Presolved: 81 rows, 185 columns, 615 nonzeros
Variable types: 175 continuous, 10 integer (10 binary)

Root relaxation: objective 4.529407e+03, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [3]:
model.printAttr('x')


    Variable            x 
-------------------------
open_factory[5]            1 
open_warehouse[4]            1 
flow_factory_warehouse[5,2]           73 
flow_factory_warehouse[5,4]         52.2 
flow_factory_warehouse[5,5]           38 
flow_warehouse_customer[1,3,1]           28 
flow_warehouse_customer[1,5,1]           13 
flow_warehouse_customer[2,1,2]            6 
flow_warehouse_customer[2,1,3]            6 
flow_warehouse_customer[2,2,1]           37 
flow_warehouse_customer[2,2,2]           10 
flow_warehouse_customer[2,3,2]           22 
flow_warehouse_customer[2,3,3]           26 
flow_warehouse_customer[2,3,4]           25 
flow_warehouse_customer[2,3,5]           24 
flow_warehouse_customer[2,4,1]           38 
flow_warehouse_customer[2,4,3]            9 
flow_warehouse_customer[2,5,1]           27 
flow_warehouse_customer[2,5,2]           35 
flow_warehouse_customer[2,5,3]           32 
flow_warehouse_customer[2,5,4]           48 
flow_warehouse_customer[2,5,5]        